<a href="https://colab.research.google.com/github/jingjieyuan573-bite/Composite_Distribution_analysis/blob/main/Composite_Monte_Carlo_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# """
# Composite Monte Carlo Simulation (Fixed, Corrected, and Sensitivity-Enhanced)

# This script runs Monte Carlo comparisons between three models:
# - Composite (center: skew-normal; tails: skew-t)
# - Skew-t
# - Skew-normal

# Features:
# - monte_carlo_sim(M,n): one Monte Carlo experiment
# - bootstrap_ci: bootstrap 95% CI for grouped means
# - run_sample_size_sensitivity: run experiments for multiple n and produce CSV + LaTeX output

# This version fixes all syntax issues and unterminated string literals and adds full sample-size sensitivity analysis with bootstrap CIs, directly displaying results.
# """

# import time
# import numpy as np
# import pandas as pd
# from scipy import stats
# import warnings
# warnings.filterwarnings('ignore')

# # ---------- RNG helper ----------

# def ensure_rng(rs=None):
#     if rs is None:
#         return np.random.RandomState(None)
#     if isinstance(rs, (int, np.integer)):
#         return np.random.RandomState(int(rs))
#     if isinstance(rs, np.random.RandomState):
#         return rs
#     return np.random.RandomState(None)

# # ---------- Skew-t sampling ----------

# def skewt_rvs(xi=0.0, omega=1.0, alpha=0.0, nu=8.0, size=1, random_state=None):
#     rng = ensure_rng(random_state)
#     U = stats.skewnorm.rvs(alpha, loc=0.0, scale=1.0, size=size, random_state=rng)
#     S = rng.chisquare(nu, size=size)
#     return xi + omega * U * np.sqrt(nu / S)

# # ---------- Composite sampling ----------

# def composite_rvs(size=1, xi=0.0, omega=1.0, alpha=15.0, nu=6.0,
#                   theta1=-2.0, theta2=2, left_frac=0.08, right_frac=0.08, random_state=None):
#     rng = ensure_rng(random_state)
#     n = int(size)
#     samples = np.empty(n, dtype=float)
#     u = rng.rand(n)
#     r1 = left_frac; r2 = right_frac
#     comp = np.where(u < r1, 0, np.where(u < r1 + (1 - r1 - r2), 1, 2))

#     for k in (0, 1, 2):
#         idx = np.where(comp == k)[0]
#         if idx.size == 0:
#             continue
#         need = idx.size
#         draws = []
#         while len(draws) < need:
#             if k == 1:
#                 batch = stats.skewnorm.rvs(alpha, loc=xi, scale=omega, size=max(need * 2, 500), random_state=rng)
#                 valid = batch[(batch > theta1) & (batch < theta2)]
#             else:
#                 batch = skewt_rvs(xi=xi, omega=omega, alpha=alpha, nu=nu, size=max(need * 2, 500), random_state=rng)
#                 if k == 0:
#                     valid = batch[batch <= theta1]
#                 else:
#                     valid = batch[batch >= theta2]
#             draws.extend(valid.tolist())
#         samples[idx] = np.array(draws[:need])
#     return samples

# # ---------- Empirical CDF factory ----------

# def empirical_cdf_factory(sample):
#     s = np.sort(np.asarray(sample))
#     n = len(s)
#     def cdf(x):
#         x = np.asarray(x)
#         return np.searchsorted(s, x, side='right') / float(n)
#     return cdf

# # ---------- Tail KS ----------

# def compute_tailks(cdf_func, data_sorted, upper_pct, lower_pct):
#     lower_val = np.percentile(data_sorted, lower_pct)
#     upper_val = np.percentile(data_sorted, upper_pct)
#     tail_points = np.concatenate([data_sorted[data_sorted <= lower_val], data_sorted[data_sorted >= upper_val]])
#     if len(tail_points) == 0:
#         return np.nan
#     fitted_vals = np.asarray(cdf_func(tail_points))
#     n = len(data_sorted)
#     empirical_vals = np.searchsorted(data_sorted, tail_points, side='right') / float(n)
#     return float(np.max(np.abs(fitted_vals - empirical_vals)))

# # ---------- Log-likelihood ----------

# def loglikelihood(model_name, x):
#     x = np.asarray(x)
#     if model_name == 'Composite':
#         ll = np.zeros_like(x, dtype=float)
#         left_mask = x <= -1
#         mid_mask = (x > -1) & (x < 0.2)
#         right_mask = x >= 0.2
#         if np.any(left_mask):
#             ll[left_mask] = stats.t.pdf(x[left_mask], df=6, loc=0, scale=1)
#         if np.any(mid_mask):
#             ll[mid_mask] = stats.skewnorm.pdf(x[mid_mask], a=15, loc=0, scale=1)
#         if np.any(right_mask):
#             ll[right_mask] = stats.t.pdf(x[right_mask], df=6, loc=0, scale=1)
#         return np.log(np.maximum(ll, 1e-12))
#     elif model_name == 'Skew-t':
#         return np.log(stats.t.pdf(x, df=5, loc=0, scale=1))
#     else:
#         return np.log(stats.skewnorm.pdf(x, a=6, loc=0, scale=1))

# # ---------- Monte Carlo simulation ----------

# def monte_carlo_sim(M=50, n=2000, seed=1234, rng=None, mc_sample_size=20000):
#     """
#     Run Monte Carlo: generate data from composite DGP and evaluate three candidate models.
#     For each replication we :
#       - draw one dataset from the composite DGP;
#       - fit simple parametric approximations to each candidate model using the dataset (so fitted CDFs reflect estimation error);
#       - draw a large parametric sample from each fitted model (mc_sample_size) to form the fitted CDF;
#       - compute TailKS between the observed data and each fitted model; compute bias/MSE from model-implied draws;
#       - compute average log-likelihood of the observed data under the fitted model.

#     This approach avoids TailKS==0 (which happens when comparing the dataset to its own empirical CDF)
#     and gives a realistic, non-degenerate comparison.
#     """
#     rng = ensure_rng(rng if rng is not None else seed)
#     rows = []

#     for rep in range(int(M)):
#         # draw one dataset from the composite DGP
#         data = composite_rvs(size=n, xi=0.0, omega=1.0, alpha=15.0, nu=6.0,
#                              theta1=-2, theta2=2, left_frac=0.08, right_frac=0.08, random_state=rng)
#         data_sorted = np.sort(data)

#         # split observed data for composite-fitting convenience
#         theta1, theta2 = -2, 2
#         left_obs = data[data <= theta1]
#         mid_obs = data[(data > theta1) & (data < theta2)]
#         right_obs = data[data >= theta2]
#         n_left, n_mid, n_right = len(left_obs), len(mid_obs), len(right_obs)

#         for model_name in ['Composite', 'Skew-t', 'Skew-normal']:
#             # Fit model parameters from the observed data (simple, robust fits)
#             try:
#                 if model_name == 'Composite':
#                     # estimate region weights
#                     r1_est = max(1e-6, n_left / float(n))
#                     r2_est = max(1e-6, n_right / float(n))
#                     rmid_est = max(1e-6, 1.0 - r1_est - r2_est)

#                     # fit center skew-normal if enough data, else fallback to canonical
#                     if n_mid >= 10:
#                         try:
#                             a_c, loc_c, scale_c = stats.skewnorm.fit(mid_obs)
#                         except Exception:
#                             a_c, loc_c, scale_c = 15.0, 0.0, 1.0
#                     else:
#                         a_c, loc_c, scale_c = 15.0, 0.0, 1.0

#                     # fit tails (t) if enough data
#                     if n_left >= 10:
#                         try:
#                             df_l, loc_l, scale_l = stats.t.fit(left_obs)
#                         except Exception:
#                             df_l, loc_l, scale_l = 6.0, 0.0, 1.0
#                     else:
#                         df_l, loc_l, scale_l = 6.0, 0.0, 1.0

#                     if n_right >= 10:
#                         try:
#                             df_r, loc_r, scale_r = stats.t.fit(right_obs)
#                         except Exception:
#                             df_r, loc_r, scale_r = 6.0, 0.0, 1.0
#                     else:
#                         df_r, loc_r, scale_r = 6.0, 0.0, 1.0

#                     # build a parametric sample from the fitted composite model
#                     n_left_s = max(1, int(mc_sample_size * r1_est))
#                     n_mid_s = max(1, int(mc_sample_size * rmid_est))
#                     n_right_s = max(1, mc_sample_size - n_left_s - n_mid_s)

#                     left_sample = stats.t.rvs(df_l, loc=loc_l, scale=scale_l, size=n_left_s, random_state=rng)
#                     mid_sample = stats.skewnorm.rvs(a_c, loc=loc_c, scale=scale_c, size=n_mid_s, random_state=rng)
#                     right_sample = stats.t.rvs(df_r, loc=loc_r, scale=scale_r, size=n_right_s, random_state=rng)

#                     param_sample = np.concatenate([left_sample, mid_sample, right_sample])

#                     # compute per-observation log-likelihood under the fitted composite
#                     ll_vals = np.empty_like(data)
#                     left_mask = data <= theta1
#                     mid_mask = (data > theta1) & (data < theta2)
#                     right_mask = data >= theta2
#                     if np.any(left_mask):
#                         ll_vals[left_mask] = stats.t.pdf(data[left_mask], df=df_l, loc=loc_l, scale=scale_l)
#                     if np.any(mid_mask):
#                         ll_vals[mid_mask] = stats.skewnorm.pdf(data[mid_mask], a=a_c, loc=loc_c, scale=scale_c)
#                     if np.any(right_mask):
#                         ll_vals[right_mask] = stats.t.pdf(data[right_mask], df=df_r, loc=loc_r, scale=scale_r)
#                     avg_loglike = float(np.mean(np.log(np.maximum(ll_vals, 1e-12))))

#                 elif model_name == 'Skew-t':
#                     # approximate skew-t by fitting a Student-t (no skew param in scipy)
#                     try:
#                         df_t, loc_t, scale_t = stats.t.fit(data)
#                     except Exception:
#                         df_t, loc_t, scale_t = 6.0, 0.0, 1.0
#                     param_sample = stats.t.rvs(df_t, loc=loc_t, scale=scale_t, size=mc_sample_size, random_state=rng)
#                     avg_loglike = float(np.mean(stats.t.logpdf(data, df=df_t, loc=loc_t, scale=scale_t)))

#                 else:  # Skew-normal
#                     try:
#                         a_sn, loc_sn, scale_sn = stats.skewnorm.fit(data)
#                     except Exception:
#                         a_sn, loc_sn, scale_sn = 6.0, 0.0, 1.0
#                     param_sample = stats.skewnorm.rvs(a_sn, loc=loc_sn, scale=scale_sn, size=mc_sample_size, random_state=rng)
#                     avg_loglike = float(np.mean(stats.skewnorm.logpdf(data, a_sn, loc=loc_sn, scale=scale_sn)))

#             except Exception as e:
#                 # fallback: use canonical parameters if any fit fails
#                 if model_name == 'Composite':
#                     param_sample = composite_rvs(size=mc_sample_size, random_state=rng)
#                     avg_loglike = float(np.mean(loglikelihood('Composite', data)))
#                 elif model_name == 'Skew-t':
#                     param_sample = skewt_rvs(size=mc_sample_size, random_state=rng)
#                     avg_loglike = float(np.mean(loglikelihood('Skew-t', data)))
#                 else:
#                     param_sample = stats.skewnorm.rvs(a=6.0, loc=0.0, scale=1.0, size=mc_sample_size, random_state=rng)
#                     avg_loglike = float(np.mean(loglikelihood('Skew-normal', data)))

#             # make fitted cdf from the parametric sample (reflects estimated parameters)
#             fitted_cdf = empirical_cdf_factory(param_sample)

#             # TailKS comparing observed data vs fitted model CDF
#             tailks = compute_tailks(fitted_cdf, data_sorted, 95, 5)

#             # compute model-implied moments (draw a size-n sample from param_sample)
#             sim_draw = rng.choice(param_sample, size=n, replace=False) if len(param_sample) >= n else rng.choice(param_sample, size=n, replace=True)
#             bias_mean = float(np.mean(sim_draw))
#             mse_mean = float(np.mean((sim_draw - 0.0) ** 2))

#             rows.append({'rep': int(rep), 'model': model_name, 'avg_loglike': avg_loglike,
#                          'tailks': tailks, 'bias_mean': bias_mean, 'mse_mean': mse_mean})

#     return pd.DataFrame(rows)

# # ---------- Bootstrap CI ----------

# def bootstrap_ci(df, model, col, B=500, alpha=0.05, rng=None):
#     rng = ensure_rng(rng)
#     vals = df[df['model'] == model][col].values
#     n = len(vals)
#     if n == 0:
#         return (np.nan, np.nan)
#     boot_means = [np.mean(rng.choice(vals, size=n, replace=True)) for _ in range(int(B))]
#     lo = np.percentile(boot_means, 100 * alpha / 2)
#     hi = np.percentile(boot_means, 100 * (1 - alpha / 2))
#     return float(lo), float(hi)

# # ---------- Run sample-size sensitivity with bootstrap ----------

# def run_sample_size_sensitivity(sample_sizes=[2000], M=500, B_boot=500, seed=1234, quick=False):
#     rng = ensure_rng(seed)
#     M_use, B_use = (max(10, int(M // 10)), max(50, int(B_boot // 10))) if quick else (int(M), int(B_boot))

#     results = []

#     for n in sample_sizes:
#         t0 = time.time()
#         print(f"\nRunning Monte Carlo simulation with sample size {n}")
#         df = monte_carlo_sim(M=M_use, n=n, seed=seed, rng=rng)

#         summary = df.groupby('model')[['avg_loglike','tailks','bias_mean','mse_mean']].mean().reset_index()
#         print("\nMonte Carlo results (averaged over repetitions):")
#         print(summary)

#         # Compute bootstrap CIs
#         for model in summary['model']:
#             ci_avgloglike = bootstrap_ci(df, model, 'avg_loglike', B=B_use, rng=rng)
#             ci_tailks = bootstrap_ci(df, model, 'tailks', B=B_use, rng=rng)
#             ci_bias = bootstrap_ci(df, model, 'bias_mean', B=B_use, rng=rng)
#             ci_mse = bootstrap_ci(df, model, 'mse_mean', B=B_use, rng=rng)
#             print(f"\n{model} 95% CI:")
#             print(f"  Avg LOGLIKE: {ci_avgloglike}")
#             print(f"  Tail KS   : {ci_tailks}")
#             print(f"  Bias      : {ci_bias}")
#             print(f"  MSE       : {ci_mse}")

#     return df, summary

# if __name__ == "__main__":
#     # 小规模测试，快速看到输出
#     df, summary = run_sample_size_sensitivity(
#         sample_sizes=[2000],  # 这里可以改回 [500,2000,5000]
#         M=50,                      # 可以改回 500
#         B_boot=100,                # 可以改回 500
#         seed=1234,
#         quick=True                 # quick=True 只做少量重复，快速看到结果
#     )



Running Monte Carlo simulation with sample size 2000


KeyboardInterrupt: 

In [4]:
"""
Composite Monte Carlo Simulation (Fixed / Self-contained)

- Fixed RNG handling and result aggregation across sample sizes.
- Saves summary CSV and LaTeX.
- Default run: n=2000, M=50 (quick small run). Adjust M, B_boot for larger experiments.
"""

import time
import numpy as np
import pandas as pd
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ---------- RNG helper ----------
def ensure_rng(rs=None):
    if rs is None:
        return np.random.RandomState(None)
    if isinstance(rs, (int, np.integer)):
        return np.random.RandomState(int(rs))
    if isinstance(rs, np.random.RandomState):
        return rs
    return np.random.RandomState(None)

# ---------- Skew-t sampling ----------
def skewt_rvs(xi=0.0, omega=1.0, alpha=0.0, nu=8.0, size=1, random_state=None):
    rng = ensure_rng(random_state)
    U = stats.skewnorm.rvs(alpha, loc=0.0, scale=1.0, size=size, random_state=rng)
    S = rng.chisquare(nu, size=size)
    return xi + omega * U * np.sqrt(nu / S)

# ---------- Composite sampling ----------
def composite_rvs(size=1, xi=0.0, omega=1.0, alpha=15.0, nu=6.0,
                  theta1=-0.3, theta2=0.3, left_frac=0.08, right_frac=0.08, random_state=None):
    rng = ensure_rng(random_state)
    n = int(size)
    samples = np.empty(n, dtype=float)
    u = rng.rand(n)
    r1 = left_frac; r2 = right_frac
    comp = np.where(u < r1, 0, np.where(u < r1 + (1 - r1 - r2), 1, 2))

    for k in (0, 1, 2):
        idx = np.where(comp == k)[0]
        if idx.size == 0:
            continue
        need = idx.size
        draws = []
        # rejection sampling batches; avoid pathological infinite loop by large batch size
        while len(draws) < need:
            if k == 1:
                batch = stats.skewnorm.rvs(alpha, loc=xi, scale=omega, size=max(need * 2, 500), random_state=rng)
                valid = batch[(batch > theta1) & (batch < theta2)]
            else:
                batch = skewt_rvs(xi=xi, omega=omega, alpha=alpha, nu=nu, size=max(need * 2, 500), random_state=rng)
                if k == 0:
                    valid = batch[batch <= theta1]
                else:
                    valid = batch[batch >= theta2]
            draws.extend(valid.tolist())
        samples[idx] = np.array(draws[:need])
    return samples

# ---------- Empirical CDF factory ----------
def empirical_cdf_factory(sample):
    s = np.sort(np.asarray(sample))
    n = len(s)
    def cdf(x):
        x = np.asarray(x)
        return np.searchsorted(s, x, side='right') / float(n)
    return cdf

# ---------- Tail KS ----------
def compute_tailks(cdf_func, data_sorted, upper_pct, lower_pct):
    lower_val = np.percentile(data_sorted, lower_pct)
    upper_val = np.percentile(data_sorted, upper_pct)
    tail_points = np.concatenate([data_sorted[data_sorted <= lower_val], data_sorted[data_sorted >= upper_val]])
    if len(tail_points) == 0:
        return np.nan
    fitted_vals = np.asarray(cdf_func(tail_points))
    n = len(data_sorted)
    empirical_vals = np.searchsorted(data_sorted, tail_points, side='right') / float(n)
    return float(np.max(np.abs(fitted_vals - empirical_vals)))

# ---------- Log-likelihood (fallback only) ----------
def loglikelihood(model_name, x):
    x = np.asarray(x)
    if model_name == 'Composite':
        ll = np.zeros_like(x, dtype=float)
        left_mask = x <= -1
        mid_mask = (x > -1) & (x < 0.2)
        right_mask = x >= 0.2
        if np.any(left_mask):
            ll[left_mask] = stats.t.pdf(x[left_mask], df=6, loc=0, scale=1)
        if np.any(mid_mask):
            ll[mid_mask] = stats.skewnorm.pdf(x[mid_mask], a=15, loc=0, scale=1)
        if np.any(right_mask):
            ll[right_mask] = stats.t.pdf(x[right_mask], df=6, loc=0, scale=1)
        return np.log(np.maximum(ll, 1e-12))
    elif model_name == 'Skew-t':
        return np.log(stats.t.pdf(x, df=5, loc=0, scale=1))
    else:
        return np.log(stats.skewnorm.pdf(x, a=6, loc=0, scale=1))

# ---------- Monte Carlo simulation ----------
def monte_carlo_sim(M=500, n=5000, seed=1, rng=None, mc_sample_size=20000):
    rng = ensure_rng(rng if rng is not None else seed)
    rows = []

    for rep in range(int(M)):
        # draw one dataset from the composite DGP
        data = composite_rvs(size=n, xi=0.0, omega=1.0, alpha=15.0, nu=6.0,
                             theta1=-0.3, theta2=0.3, left_frac=0.08, right_frac=0.08, random_state=rng)
        data_sorted = np.sort(data)

        # split observed data for composite-fitting convenience
        theta1, theta2 = -0.3, 0.3
        left_obs = data[data <= theta1]
        mid_obs = data[(data > theta1) & (data < theta2)]
        right_obs = data[data >= theta2]
        n_left, n_mid, n_right = len(left_obs), len(mid_obs), len(right_obs)

        for model_name in ['Composite', 'Skew-t', 'Skew-normal']:
            try:
                if model_name == 'Composite':
                    # estimate region weights
                    r1_est = max(1e-6, n_left / float(n))
                    r2_est = max(1e-6, n_right / float(n))
                    rmid_est = max(1e-6, 1.0 - r1_est - r2_est)

                    # fit center skew-normal if enough data, else fallback to canonical
                    if n_mid >= 10:
                        try:
                            a_c, loc_c, scale_c = stats.skewnorm.fit(mid_obs)
                        except Exception:
                            a_c, loc_c, scale_c = 15.0, 0.0, 1.0
                    else:
                        a_c, loc_c, scale_c = 15.0, 0.0, 1.0

                    # fit tails (t) if enough data
                    if n_left >= 10:
                        try:
                            df_l, loc_l, scale_l = stats.t.fit(left_obs)
                        except Exception:
                            df_l, loc_l, scale_l = 6.0, 0.0, 1.0
                    else:
                        df_l, loc_l, scale_l = 6.0, 0.0, 1.0

                    if n_right >= 10:
                        try:
                            df_r, loc_r, scale_r = stats.t.fit(right_obs)
                        except Exception:
                            df_r, loc_r, scale_r = 6.0, 0.0, 1.0
                    else:
                        df_r, loc_r, scale_r = 6.0, 0.0, 1.0

                    # build a parametric sample from the fitted composite model
                    n_left_s = max(1, int(mc_sample_size * r1_est))
                    n_mid_s = max(1, int(mc_sample_size * rmid_est))
                    n_right_s = max(1, mc_sample_size - n_left_s - n_mid_s)

                    left_sample = stats.t.rvs(df_l, loc=loc_l, scale=scale_l, size=n_left_s, random_state=rng)
                    mid_sample = stats.skewnorm.rvs(a_c, loc=loc_c, scale=scale_c, size=n_mid_s, random_state=rng)
                    right_sample = stats.t.rvs(df_r, loc=loc_r, scale=scale_r, size=n_right_s, random_state=rng)

                    param_sample = np.concatenate([left_sample, mid_sample, right_sample])

                    # compute per-observation log-likelihood under the fitted composite
                    ll_vals = np.empty_like(data)
                    left_mask = data <= theta1
                    mid_mask = (data > theta1) & (data < theta2)
                    right_mask = data >= theta2
                    if np.any(left_mask):
                        ll_vals[left_mask] = stats.t.pdf(data[left_mask], df=df_l, loc=loc_l, scale=scale_l)
                    if np.any(mid_mask):
                        ll_vals[mid_mask] = stats.skewnorm.pdf(data[mid_mask], a=a_c, loc=loc_c, scale=scale_c)
                    if np.any(right_mask):
                        ll_vals[right_mask] = stats.t.pdf(data[right_mask], df=df_r, loc=loc_r, scale=scale_r)
                    avg_loglike = float(np.mean(np.log(np.maximum(ll_vals, 1e-12))))

                elif model_name == 'Skew-t':
                    # approximate skew-t by fitting a Student-t (no skew param in scipy)
                    try:
                        df_t, loc_t, scale_t = stats.t.fit(data)
                    except Exception:
                        df_t, loc_t, scale_t = 6.0, 0.0, 1.0
                    param_sample = stats.t.rvs(df_t, loc=loc_t, scale=scale_t, size=mc_sample_size, random_state=rng)
                    avg_loglike = float(np.mean(stats.t.logpdf(data, df=df_t, loc=loc_t, scale=scale_t)))

                else:  # Skew-normal
                    try:
                        a_sn, loc_sn, scale_sn = stats.skewnorm.fit(data)
                    except Exception:
                        a_sn, loc_sn, scale_sn = 6.0, 0.0, 1.0
                    param_sample = stats.skewnorm.rvs(a_sn, loc=loc_sn, scale=scale_sn, size=mc_sample_size, random_state=rng)
                    avg_loglike = float(np.mean(stats.skewnorm.logpdf(data, a_sn, loc=loc_sn, scale=scale_sn)))

            except Exception as e:
                # fallback: use canonical parameters if any fit fails
                if model_name == 'Composite':
                    param_sample = composite_rvs(size=mc_sample_size, random_state=rng)
                    avg_loglike = float(np.mean(loglikelihood('Composite', data)))
                elif model_name == 'Skew-t':
                    param_sample = skewt_rvs(size=mc_sample_size, random_state=rng)
                    avg_loglike = float(np.mean(loglikelihood('Skew-t', data)))
                else:
                    param_sample = stats.skewnorm.rvs(a=6.0, loc=0.0, scale=1.0, size=mc_sample_size, random_state=rng)
                    avg_loglike = float(np.mean(loglikelihood('Skew-normal', data)))

            # make fitted cdf from the parametric sample (reflects estimated parameters)
            fitted_cdf = empirical_cdf_factory(param_sample)

            # TailKS comparing observed data vs fitted model CDF
            tailks = compute_tailks(fitted_cdf, data_sorted, 95, 5)

            # compute model-implied moments (draw a size-n sample from param_sample)
            sim_draw = rng.choice(param_sample, size=n, replace=False) if len(param_sample) >= n else rng.choice(param_sample, size=n, replace=True)
            bias_mean = float(np.mean(sim_draw))
            mse_mean = float(np.mean((sim_draw - 0.0) ** 2))

            rows.append({'rep': int(rep), 'model': model_name, 'avg_loglike': avg_loglike,
                         'tailks': tailks, 'bias_mean': bias_mean, 'mse_mean': mse_mean})

    return pd.DataFrame(rows)

# ---------- Bootstrap CI ----------
def bootstrap_ci(df, model, col, B=500, alpha=0.05, rng=None):
    rng = ensure_rng(rng)
    vals = df[df['model'] == model][col].values
    n = len(vals)
    if n == 0:
        return (np.nan, np.nan)
    boot_means = [np.mean(rng.choice(vals, size=n, replace=True)) for _ in range(int(B))]
    lo = np.percentile(boot_means, 100 * alpha / 2)
    hi = np.percentile(boot_means, 100 * (1 - alpha / 2))
    return float(lo), float(hi)

# ---------- Run sample-size sensitivity with bootstrap ----------
def run_sample_size_sensitivity(sample_sizes=[5000], M=500, B_boot=500, seed=1234, quick=False, mc_sample_size=20000, out_prefix="mc_results"):
    rng_master = ensure_rng(seed)
    if quick:
        M_use = max(10, int(M // 10))
        B_use = max(50, int(B_boot // 10))
    else:
        M_use, B_use = int(M), int(B_boot)

    all_rows = []
    summaries = []

    for n in sample_sizes:
        t0 = time.time()
        print(f"\nRunning Monte Carlo simulation with sample size {n} (M={M_use}, B_boot={B_use})")
        # derive per-n RNG for reproducibility
        rng_n = ensure_rng(rng_master.randint(2**31 - 1))

        df_n = monte_carlo_sim(M=M_use, n=n, seed=None, rng=rng_n, mc_sample_size=mc_sample_size)
        df_n['n'] = int(n)
        all_rows.append(df_n)

        # compute mean summary per model
        summary = df_n.groupby('model')[['avg_loglike','tailks','bias_mean','mse_mean']].mean().reset_index()

        for model in summary['model']:
            ci_rng = ensure_rng(rng_n.randint(2**31 - 1))
            ci_avgloglike = bootstrap_ci(df_n, model, 'avg_loglike', B=B_use, rng=ci_rng)
            ci_tailks     = bootstrap_ci(df_n, model, 'tailks', B=B_use, rng=ci_rng)
            ci_bias       = bootstrap_ci(df_n, model, 'bias_mean', B=B_use, rng=ci_rng)
            ci_mse        = bootstrap_ci(df_n, model, 'mse_mean', B=B_use, rng=ci_rng)

            summaries.append({
                'n': int(n),
                'model': model,
                'avg_loglike_mean': float(summary.loc[summary['model'] == model, 'avg_loglike'].values[0]),
                'avg_loglike_lo': ci_avgloglike[0],
                'avg_loglike_hi': ci_avgloglike[1],
                'tailks_mean': float(summary.loc[summary['model'] == model, 'tailks'].values[0]),
                'tailks_lo': ci_tailks[0],
                'tailks_hi': ci_tailks[1],
                'bias_mean': float(summary.loc[summary['model'] == model, 'bias_mean'].values[0]),
                'bias_lo': ci_bias[0],
                'bias_hi': ci_bias[1],
                'mse_mean': float(summary.loc[summary['model'] == model, 'mse_mean'].values[0]),
                'mse_lo': ci_mse[0],
                'mse_hi': ci_mse[1],
            })

            print(f"\n{model} (n={n}) 95% CI:")
            print(f"  Avg LOGLIKE: {ci_avgloglike}")
            print(f"  Tail KS   : {ci_tailks}")
            print(f"  Bias      : {ci_bias}")
            print(f"  MSE       : {ci_mse}")

        print(f"Finished n={n} in {time.time() - t0:.1f}s")

    df_all = pd.concat(all_rows, ignore_index=True) if len(all_rows) > 0 else pd.DataFrame()
    df_summary = pd.DataFrame(summaries)

    csv_file = f"{out_prefix}_summary.csv"
    tex_file = f"{out_prefix}_summary.tex"
    df_summary.to_csv(csv_file, index=False)
    try:
        with open(tex_file, "w", encoding="utf-8") as f:
            f.write(df_summary.to_latex(index=False, float_format="%.6f"))
    except Exception as e:
        print("Warning: could not write LaTeX file:", e)

    print(f"\nSaved summary CSV to {csv_file} and LaTeX to {tex_file} (if no error).")
    return df_all, df_summary

# ---------- Main ----------
if __name__ == "__main__":
    # 仅跑一次 n=2000（快速示例）。若要更稳定结果，请把 quick=False 并把 M, B_boot 调高。
    df_all, df_summary = run_sample_size_sensitivity(
        sample_sizes=[5000],
        M=500,           # replication 数，真实研究建议 >=200-500
        B_boot=500,     # bootstrap 次数，建议 >=500
        seed=1234,
        quick=False,    # quick=True 会把 M 和 B_boot 缩小为约1/10
        mc_sample_size=20000,
        out_prefix="mc_composite_n2000"
    )

    print("\nFinal summary:")
    print(df_summary)




#Running Monte Carlo simulation with sample size 500 (M=50, B_boot=100)

#Composite (n=500) 95% CI:
#  Avg LOGLIKE: (1.0203065271887517, 1.0425575349824716)
#  Tail KS   : (0.009981749999999996, 0.01109264999999999)
#  Bias      : (0.061599514988977935, 0.07741750208248337)
#  MSE       : (0.23937460912980368, 0.3048096085471503)

#Skew-normal (n=500) 95% CI:
#  Avg LOGLIKE: (-0.7403803231835338, -0.7020945558841764)
#  Tail KS   : (0.06512174999999998, 0.08911767499999997)
#  Bias      : (0.05227484264304224, 0.0788688540982724)
#  MSE       : (0.25106083445145266, 0.2771680613256963)

#Skew-t (n=500) 95% CI:
#  Avg LOGLIKE: (0.18602374445674738, 0.23045145733729883)
#  Tail KS   : (0.030686700000000004, 0.03213375)
#  Bias      : (0.17429622197376368, 0.7717864156751897)
#  MSE       : (36.180392310986164, 1741.1327376810939)
#Finished n=500 in 12297.8s

#Saved summary CSV to mc_composite_n2000_summary.csv and LaTeX to mc_composite_n2000_summary.tex (if no error).

#Final summary:
#     n        model  avg_loglike_mean  avg_loglike_lo  avg_loglike_hi  \
#0  500    Composite          1.031592        1.020307        1.042558
#1  500  Skew-normal         -0.723335       -0.740380       -0.702095
#2  500       Skew-t          0.209404        0.186024        0.230451

 #  tailks_mean  tailks_lo  tailks_hi  bias_mean   bias_lo   bias_hi  \
#0     0.010585   0.009982   0.011093   0.068968  0.061600  0.077418
#1     0.076773   0.065122   0.089118   0.065245  0.052275  0.078869
#2     0.031470   0.030687   0.032134   0.402457  0.174296  0.771786

 #    mse_mean     mse_lo       mse_hi
#0    0.268879   0.239375     0.304810
#1    0.263083   0.251061     0.277168
#2  644.175063  36.180392  1741.132738


Running Monte Carlo simulation with sample size 5000 (M=500, B_boot=500)


KeyboardInterrupt: 

In [9]:
# """
# Monte Carlo Simulation (Composite, Fixed Thresholds, Parallel)

# - theta1=-0.3, theta2=0.3 only
# - M=500, B_boot=500
# - Parallelized for faster computation
# - Outputs CSV and LaTeX summary
# """

# import time
# import numpy as np
# import pandas as pd
# from scipy import stats
# import warnings
# from joblib import Parallel, delayed
# warnings.filterwarnings('ignore')

# # ---------- RNG helper ----------
# def ensure_rng(rs=None):
#     if rs is None:
#         return np.random.RandomState(None)
#     if isinstance(rs, (int, np.integer)):
#         return np.random.RandomState(int(rs))
#     if isinstance(rs, np.random.RandomState):
#         return rs
#     return np.random.RandomState(None)

# # ---------- Skew-t sampling ----------
# def skewt_rvs(xi=0.0, omega=1.0, alpha=0.0, nu=6.0, size=1, random_state=None):
#     rng = ensure_rng(random_state)
#     U = stats.skewnorm.rvs(alpha, loc=0.0, scale=1.0, size=size, random_state=rng)
#     S = rng.chisquare(nu, size=size)
#     return xi + omega * U * np.sqrt(nu / S)

# # ---------- Composite sampling ----------
# def composite_rvs(size=1, xi=0.0, omega=1.0, alpha=15.0, nu=6.0,
#                   theta1=-0.3, theta2=0.3, left_frac=0.08, right_frac=0.08, random_state=None):
#     rng = ensure_rng(random_state)
#     n = int(size)
#     samples = np.empty(n, dtype=float)
#     u = rng.rand(n)
#     r1 = left_frac; r2 = right_frac
#     comp = np.where(u < r1, 0, np.where(u < r1 + (1 - r1 - r2), 1, 2))

#     for k in (0, 1, 2):
#         idx = np.where(comp == k)[0]
#         if idx.size == 0:
#             continue
#         need = idx.size
#         draws = []
#         while len(draws) < need:
#             if k == 1:
#                 batch = stats.skewnorm.rvs(alpha, loc=xi, scale=omega, size=max(need*2, 500), random_state=rng)
#                 valid = batch[(batch > theta1) & (batch < theta2)]
#             else:
#                 batch = skewt_rvs(xi=xi, omega=omega, alpha=alpha, nu=nu, size=max(need*2, 500), random_state=rng)
#                 if k == 0:
#                     valid = batch[batch <= theta1]
#                 else:
#                     valid = batch[batch >= theta2]
#             draws.extend(valid.tolist())
#         samples[idx] = np.array(draws[:need])
#     return samples

# # ---------- Empirical CDF ----------
# def empirical_cdf_factory(sample):
#     s = np.sort(np.asarray(sample))
#     n = len(s)
#     def cdf(x):
#         x = np.asarray(x)
#         return np.searchsorted(s, x, side='right') / float(n)
#     return cdf

# # ---------- Tail KS ----------
# def compute_tailks(cdf_func, data_sorted, upper_pct=95, lower_pct=5):
#     lower_val = np.percentile(data_sorted, lower_pct)
#     upper_val = np.percentile(data_sorted, upper_pct)
#     tail_points = np.concatenate([data_sorted[data_sorted <= lower_val], data_sorted[data_sorted >= upper_val]])
#     if len(tail_points) == 0:
#         return np.nan
#     fitted_vals = np.asarray(cdf_func(tail_points))
#     n = len(data_sorted)
#     empirical_vals = np.searchsorted(data_sorted, tail_points, side='right') / float(n)
#     return float(np.max(np.abs(fitted_vals - empirical_vals)))

# # ---------- Bootstrap CI (parallel) ----------
# def bootstrap_ci(vals, B=500, alpha=0.05, rng_seed=None):
#     rng = np.random.RandomState(rng_seed)
#     n = len(vals)
#     boot_means = Parallel(n_jobs=-1)(delayed(lambda r: np.mean(rng.choice(vals, size=n, replace=True)))(i) for i in range(B))
#     lo = np.percentile(boot_means, 100*alpha/2)
#     hi = np.percentile(boot_means, 100*(1-alpha/2))
#     return float(lo), float(hi)

# # ---------- Monte Carlo for one replication ----------
# def monte_carlo_one(rep, n, mc_sample_size, seed=None):
#     rng = np.random.RandomState(seed + rep if seed is not None else None)
#     data = composite_rvs(size=n, theta1=-0.3, theta2=0.3, random_state=rng)
#     data_sorted = np.sort(data)
#     results = []

#     for model_name in ['Composite', 'Skew-normal', 'Skew-t']:
#         try:
#             if model_name == 'Composite':
#                 theta1, theta2 = -0.3, 0.3
#                 left_obs = data[data <= theta1]
#                 mid_obs = data[(data > theta1) & (data < theta2)]
#                 right_obs = data[data >= theta2]

#                 # fit skew-normal to center
#                 if len(mid_obs) >= 10:
#                     a_c, loc_c, scale_c = stats.skewnorm.fit(mid_obs)
#                 else:
#                     a_c, loc_c, scale_c = 15.0, 0.0, 1.0
#                 # fit t to tails
#                 df_l, loc_l, scale_l = stats.t.fit(left_obs) if len(left_obs)>=10 else (6.0, 0.0, 1.0)
#                 df_r, loc_r, scale_r = stats.t.fit(right_obs) if len(right_obs)>=10 else (6.0, 0.0, 1.0)

#                 # parametric sample
#                 n_left_s = max(1, int(mc_sample_size*len(left_obs)/n))
#                 n_mid_s = max(1, int(mc_sample_size*len(mid_obs)/n))
#                 n_right_s = max(1, mc_sample_size - n_left_s - n_mid_s)

#                 left_sample = stats.t.rvs(df_l, loc=loc_l, scale=scale_l, size=n_left_s, random_state=rng)
#                 mid_sample = stats.skewnorm.rvs(a_c, loc=loc_c, scale=scale_c, size=n_mid_s, random_state=rng)
#                 right_sample = stats.t.rvs(df_r, loc=loc_r, scale=scale_r, size=n_right_s, random_state=rng)
#                 param_sample = np.concatenate([left_sample, mid_sample, right_sample])

#                 ll_vals = np.empty_like(data)
#                 ll_vals[data <= theta1] = stats.t.pdf(data[data <= theta1], df=df_l, loc=loc_l, scale=scale_l)
#                 ll_vals[(data>theta1)&(data<theta2)] = stats.skewnorm.pdf(data[(data>theta1)&(data<theta2)], a=a_c, loc=loc_c, scale=scale_c)
#                 ll_vals[data >= theta2] = stats.t.pdf(data[data >= theta2], df=df_r, loc=loc_r, scale=scale_r)
#                 avg_loglike = float(np.mean(np.log(np.maximum(ll_vals,1e-12))))

#             elif model_name == 'Skew-normal':
#                 a_sn, loc_sn, scale_sn = stats.skewnorm.fit(data)
#                 param_sample = stats.skewnorm.rvs(a_sn, loc=loc_sn, scale=scale_sn, size=mc_sample_size, random_state=rng)
#                 avg_loglike = float(np.mean(stats.skewnorm.logpdf(data, a_sn, loc=loc_sn, scale=scale_sn)))

#             else: # Skew-t
#                 df_t, loc_t, scale_t = stats.t.fit(data)
#                 param_sample = stats.t.rvs(df_t, loc=loc_t, scale=scale_t, size=mc_sample_size, random_state=rng)
#                 avg_loglike = float(np.mean(stats.t.logpdf(data, df=df_t, loc=loc_t, scale=scale_t)))

#             fitted_cdf = empirical_cdf_factory(param_sample)
#             tailks = compute_tailks(fitted_cdf, data_sorted)
#             sim_draw = rng.choice(param_sample, size=n, replace=False if len(param_sample)>=n else True)
#             bias_mean = float(np.mean(sim_draw))
#             mse_mean = float(np.mean((sim_draw - 0.0)**2))
#         except Exception as e:
#             avg_loglike, tailks, bias_mean, mse_mean = np.nan, np.nan, np.nan, np.nan

#         results.append({'rep':rep, 'model':model_name, 'avg_loglike':avg_loglike,
#                         'tailks':tailks, 'bias_mean':bias_mean, 'mse_mean':mse_mean})
#     return results

# # ---------- Run Monte Carlo ----------
# def run_monte_carlo_parallel(n=500, M=500, B_boot=500, mc_sample_size=20000, seed=1234, out_prefix="mc_composite"):
#     t0 = time.time()
#     print(f"Running Monte Carlo: n={n}, M={M}, B_boot={B_boot}")

#     all_results = Parallel(n_jobs=-1)(delayed(monte_carlo_one)(rep, n, mc_sample_size, seed) for rep in range(M))
#     df_all = pd.DataFrame([r for sublist in all_results for r in sublist])

#     # compute bootstrap CI per model
#     summaries = []
#     for model in df_all['model'].unique():
#         df_model = df_all[df_all['model']==model]
#         ci_rng_seed = seed
#         avg_loglike_lo, avg_loglike_hi = bootstrap_ci(df_model['avg_loglike'].values, B=B_boot, rng_seed=ci_rng_seed)
#         tailks_lo, tailks_hi = bootstrap_ci(df_model['tailks'].values, B=B_boot, rng_seed=ci_rng_seed+1)
#         bias_lo, bias_hi = bootstrap_ci(df_model['bias_mean'].values, B=B_boot, rng_seed=ci_rng_seed+2)
#         mse_lo, mse_hi = bootstrap_ci(df_model['mse_mean'].values, B=B_boot, rng_seed=ci_rng_seed+3)

#         summaries.append({
#             'n': n,
#             'model': model,
#             'avg_loglike_mean': df_model['avg_loglike'].mean(),
#             'avg_loglike_lo': avg_loglike_lo,
#             'avg_loglike_hi': avg_loglike_hi,
#             'tailks_mean': df_model['tailks'].mean(),
#             'tailks_lo': tailks_lo,
#             'tailks_hi': tailks_hi,
#             'bias_mean': df_model['bias_mean'].mean(),
#             'bias_lo': bias_lo,
#             'bias_hi': bias_hi,
#             'mse_mean': df_model['mse_mean'].mean(),
#             'mse_lo': mse_lo,
#             'mse_hi': mse_hi
#         })
#         print(f"\n{model} (n={n}) 95% CI:")
#         print(f"  Avg LOGLIKE: ({avg_loglike_lo:.6f}, {avg_loglike_hi:.6f})")
#         print(f"  Tail KS   : ({tailks_lo:.6f}, {tailks_hi:.6f})")
#         print(f"  Bias      : ({bias_lo:.6f}, {bias_hi:.6f})")
#         print(f"  MSE       : ({mse_lo:.6f}, {mse_hi:.6f})")

#     df_summary = pd.DataFrame(summaries)
#     csv_file = f"{out_prefix}_summary.csv"
#     tex_file = f"{out_prefix}_summary.tex"
#     df_summary.to_csv(csv_file, index=False)
#     with open(tex_file, "w", encoding="utf-8") as f:
#         f.write(df_summary.to_latex(index=False, float_format="%.6f"))

#     print(f"\nFinished Monte Carlo in {time.time()-t0:.1f}s")
#     print(f"Saved summary CSV to {csv_file} and LaTeX to {tex_file}")
#     return df_all, df_summary

# # ---------- Main ----------
# if __name__ == "__main__":
#     df_all, df_summary = run_monte_carlo_parallel(n=5000, M=200, B_boot=500, mc_sample_size=20000, seed=1234)
#     print("\nFinal summary:")
#     print(df_summary)


Running Monte Carlo: n=5000, M=200, B_boot=500

Composite (n=5000) 95% CI:
  Avg LOGLIKE: (0.819865, 0.823289)
  Tail KS   : (0.007623, 0.008518)
  Bias      : (0.175823, 0.177596)
  MSE       : (0.169777, 0.188341)

Skew-normal (n=5000) 95% CI:
  Avg LOGLIKE: (-0.321716, -0.312322)
  Tail KS   : (0.033702, 0.036747)
  Bias      : (0.223033, 0.225862)
  MSE       : (0.172196, 0.175756)

Skew-t (n=5000) 95% CI:
  Avg LOGLIKE: (0.119527, 0.123650)
  Tail KS   : (0.022955, 0.023537)
  Bias      : (0.146862, 0.160341)
  MSE       : (2.258317, 27.070876)

Finished Monte Carlo in 560.5s
Saved summary CSV to mc_composite_summary.csv and LaTeX to mc_composite_summary.tex

Final summary:
      n        model  avg_loglike_mean  avg_loglike_lo  avg_loglike_hi  \
0  5000    Composite          0.821257        0.819865        0.823289   
1  5000  Skew-normal         -0.317281       -0.321716       -0.312322   
2  5000       Skew-t          0.121917        0.119527        0.123650   

   tailks_mean 

In [12]:
"""
Monte Carlo Simulation (Composite, Skew-normal, Skew-t, Parallel)
for large n M B
- theta1=-0.3, theta2=0.3 only
- M=500, B_boot=500
- Parallelized for faster computation
- Outputs CSV and LaTeX summary
"""

import time
import numpy as np
import pandas as pd
from scipy import stats
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

# ---------- RNG helper ----------
def ensure_rng(rs=None):
    if rs is None:
        return np.random.RandomState(None)
    if isinstance(rs, (int, np.integer)):
        return np.random.RandomState(int(rs))
    if isinstance(rs, np.random.RandomState):
        return rs
    return np.random.RandomState(None)

# ---------- Composite sampling ----------
def composite_rvs(size=1, theta1=-0.3, theta2=0.3, left_frac=0.08, right_frac=0.08, random_state=None):
    rng = ensure_rng(random_state)
    n = int(size)
    samples = np.empty(n, dtype=float)
    u = rng.rand(n)
    r1, r2 = left_frac, right_frac
    comp = np.where(u < r1, 0, np.where(u < 1-r2, 1, 2))
    for k in (0,1,2):
        idx = np.where(comp==k)[0]
        if idx.size==0: continue
        need = idx.size
        draws = []
        while len(draws)<need:
            if k==1:
                batch = stats.uniform.rvs(loc=theta1, scale=theta2-theta1, size=max(need*2,500), random_state=rng)
                valid = batch[(batch>theta1) & (batch<theta2)]
            else:
                batch = stats.t.rvs(df=6, loc=0, scale=1, size=max(need*2,500), random_state=rng)
                valid = batch[batch<=theta1] if k==0 else batch[batch>=theta2]
            draws.extend(valid.tolist())
        samples[idx] = np.array(draws[:need])
    return samples

# ---------- Empirical CDF ----------
def empirical_cdf_factory(sample):
    s = np.sort(np.asarray(sample))
    n = len(s)
    def cdf(x):
        x = np.asarray(x)
        return np.searchsorted(s, x, side='right')/float(n)
    return cdf

# ---------- Tail KS ----------
def compute_tailks(cdf_func, data_sorted, upper_pct=95, lower_pct=5):
    lower_val = np.percentile(data_sorted, lower_pct)
    upper_val = np.percentile(data_sorted, upper_pct)
    tail_points = np.concatenate([data_sorted[data_sorted<=lower_val], data_sorted[data_sorted>=upper_val]])
    if len(tail_points)==0: return np.nan
    fitted_vals = np.asarray(cdf_func(tail_points))
    n = len(data_sorted)
    empirical_vals = np.searchsorted(data_sorted, tail_points, side='right')/float(n)
    return float(np.max(np.abs(fitted_vals-empirical_vals)))

# ---------- Bootstrap CI ----------
def bootstrap_ci(vals, B=500, alpha=0.05, rng_seed=None):
    rng = np.random.RandomState(rng_seed)
    n = len(vals)
    boot_means = Parallel(n_jobs=-1)(delayed(lambda i: np.mean(rng.choice(vals, size=n, replace=True)))(i) for i in range(B))
    lo = np.percentile(boot_means, 100*alpha/2)
    hi = np.percentile(boot_means, 100*(1-alpha/2))
    return float(lo), float(hi)

# ---------- One Monte Carlo replication ----------
def monte_carlo_one(rep, n, mc_sample_size, seed=None):
    rng = np.random.RandomState(seed+rep if seed is not None else None)
    data = composite_rvs(size=n, theta1=-0.3, theta2=0.3, random_state=rng)
    data_sorted = np.sort(data)
    results = []

    for model_name in ['Composite','Skew-normal','Skew-t']:
        try:
            if model_name=='Composite':
                theta1,theta2=-0.3,0.3
                left_obs = data[data<=theta1]
                mid_obs = data[(data>theta1)&(data<theta2)]
                right_obs = data[data>=theta2]
                # fit skew-normal for center
                if len(mid_obs)>=10:
                    a_c, loc_c, scale_c = stats.skewnorm.fit(mid_obs)
                else:
                    a_c, loc_c, scale_c = 0.0,0.0,1.0
                # fit t for tails
                df_l, loc_l, scale_l = stats.t.fit(left_obs) if len(left_obs)>=10 else (6.0,0.0,1.0)
                df_r, loc_r, scale_r = stats.t.fit(right_obs) if len(right_obs)>=10 else (6.0,0.0,1.0)
                # parametric sample
                n_left_s = max(1,int(mc_sample_size*len(left_obs)/n))
                n_mid_s = max(1,int(mc_sample_size*len(mid_obs)/n))
                n_right_s = max(1,mc_sample_size-n_left_s-n_mid_s)
                left_sample = stats.t.rvs(df_l, loc=loc_l, scale=scale_l, size=n_left_s, random_state=rng)
                mid_sample = stats.skewnorm.rvs(a_c, loc=loc_c, scale=scale_c, size=n_mid_s, random_state=rng)
                right_sample = stats.t.rvs(df_r, loc=loc_r, scale=scale_r, size=n_right_s, random_state=rng)
                param_sample = np.concatenate([left_sample,mid_sample,right_sample])
                # log-likelihood
                ll_vals = np.empty_like(data)
                ll_vals[data<=theta1] = stats.t.pdf(data[data<=theta1], df=df_l, loc=loc_l, scale=scale_l)
                ll_vals[(data>theta1)&(data<theta2)] = stats.skewnorm.pdf(data[(data>theta1)&(data<theta2)], a_c, loc_c, scale_c)
                ll_vals[data>=theta2] = stats.t.pdf(data[data>=theta2], df=df_r, loc=loc_r, scale=scale_r)
                avg_loglike = float(np.mean(np.log(np.maximum(ll_vals,1e-12))))
            elif model_name=='Skew-normal':
                a_sn, loc_sn, scale_sn = stats.skewnorm.fit(data)
                param_sample = stats.skewnorm.rvs(a_sn, loc=loc_sn, scale=scale_sn, size=mc_sample_size, random_state=rng)
                avg_loglike = float(np.mean(stats.skewnorm.logpdf(data, a_sn, loc_sn, scale_sn)))
            else: # Skew-t
                df_t, loc_t, scale_t = stats.t.fit(data)
                param_sample = stats.t.rvs(df_t, loc=loc_t, scale=scale_t, size=mc_sample_size, random_state=rng)
                avg_loglike = float(np.mean(stats.t.logpdf(data, df=df_t, loc=loc_t, scale=scale_t)))

            fitted_cdf = empirical_cdf_factory(param_sample)
            tailks = compute_tailks(fitted_cdf, data_sorted)
            sim_draw = rng.choice(param_sample, size=n, replace=False if len(param_sample)>=n else True)
            bias_mean = float(np.mean(sim_draw))
            mse_mean = float(np.mean((sim_draw-0.0)**2))
        except:
            avg_loglike, tailks, bias_mean, mse_mean = np.nan,np.nan,np.nan,np.nan
        results.append({'rep':rep,'model':model_name,'avg_loglike':avg_loglike,
                        'tailks':tailks,'bias_mean':bias_mean,'mse_mean':mse_mean})
    return results

# ---------- Run Monte Carlo ----------
def run_monte_carlo_parallel(n=5000, M=500, B_boot=500, mc_sample_size=20000, seed=1234, out_prefix="mc_composite"):
    t0 = time.time()
    print(f"Running Monte Carlo: n={n}, M={M}, B_boot={B_boot}")
    all_results = Parallel(n_jobs=-1)(delayed(monte_carlo_one)(rep,n,mc_sample_size,seed) for rep in range(M))
    df_all = pd.DataFrame([r for sublist in all_results for r in sublist])
    # compute bootstrap CI per model
    summaries = []
    for model in df_all['model'].unique():
        df_model = df_all[df_all['model']==model]
        avg_loglike_lo, avg_loglike_hi = bootstrap_ci(df_model['avg_loglike'].values, B=B_boot, rng_seed=seed)
        tailks_lo, tailks_hi = bootstrap_ci(df_model['tailks'].values, B=B_boot, rng_seed=seed+1)
        bias_lo, bias_hi = bootstrap_ci(df_model['bias_mean'].values, B=B_boot, rng_seed=seed+2)
        mse_lo, mse_hi = bootstrap_ci(df_model['mse_mean'].values, B=B_boot, rng_seed=seed+3)
        summaries.append({
            'n': n, 'model': model,
            'avg_loglike_mean': df_model['avg_loglike'].mean(),
            'avg_loglike_lo': avg_loglike_lo,
            'avg_loglike_hi': avg_loglike_hi,
            'tailks_mean': df_model['tailks'].mean(),
            'tailks_lo': tailks_lo,
            'tailks_hi': tailks_hi,
            'bias_mean': df_model['bias_mean'].mean(),
            'bias_lo': bias_lo,
            'bias_hi': bias_hi,
            'mse_mean': df_model['mse_mean'].mean(),
            'mse_lo': mse_lo,
            'mse_hi': mse_hi
        })
        print(f"\n{model} (n={n}) 95% CI:")
        print(f"  Avg LOGLIKE: ({avg_loglike_lo:.6f}, {avg_loglike_hi:.6f})")
        print(f"  Tail KS   : ({tailks_lo:.6f}, {tailks_hi:.6f})")
        print(f"  Bias      : ({bias_lo:.6f}, {bias_hi:.6f})")
        print(f"  MSE       : ({mse_lo:.6f}, {mse_hi:.6f})")

    df_summary = pd.DataFrame(summaries)
    csv_file = f"{out_prefix}_summary.csv"
    tex_file = f"{out_prefix}_summary.tex"
    df_summary.to_csv(csv_file,index=False)
    with open(tex_file,"w",encoding="utf-8") as f:
        f.write(df_summary.to_latex(index=False,float_format="%.6f"))
    print(f"\nFinished Monte Carlo in {time.time()-t0:.1f}s")
    print(f"Saved summary CSV to {csv_file} and LaTeX to {tex_file}")
    return df_all, df_summary

# ---------- Main ----------
if __name__=="__main__":
    df_all, df_summary = run_monte_carlo_parallel(n=5000, M=500, B_boot=500, mc_sample_size=20000, seed=1234)
    print("\nFinal summary:")
    print(df_summary)


Running Monte Carlo: n=5000, M=500, B_boot=500

Composite (n=5000) 95% CI:
  Avg LOGLIKE: (0.110033, 0.112461)
  Tail KS   : (0.006635, 0.006803)
  Bias      : (-0.000970, 0.000582)
  MSE       : (0.284660, 0.294595)

Skew-normal (n=5000) 95% CI:
  Avg LOGLIKE: (-0.863804, -0.856924)
  Tail KS   : (0.052294, 0.054251)
  Bias      : (-0.001483, 0.000946)
  MSE       : (0.327104, 0.330715)

Skew-t (n=5000) 95% CI:
  Avg LOGLIKE: (-0.408264, -0.403928)
  Tail KS   : (0.015666, 0.015915)
  Bias      : (-0.002386, 0.002377)
  MSE       : (1.500749, 7.261746)

Finished Monte Carlo in 244.2s
Saved summary CSV to mc_composite_summary.csv and LaTeX to mc_composite_summary.tex

Final summary:
      n        model  avg_loglike_mean  avg_loglike_lo  avg_loglike_hi  \
0  5000    Composite          0.111351        0.110033        0.112461   
1  5000  Skew-normal         -0.859650       -0.863804       -0.856924   
2  5000       Skew-t         -0.405693       -0.408264       -0.403928   

   tailks_m